In [1]:
import os
import json
import dicttoxml
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec
from xml.dom.minidom import parseString

# 환경 변수 설정
# os.environ['PINECONE_API_KEY'] = 'your_actual_pinecone_api_key_here'
api_key = os.getenv('PINECONE_API_KEY')
if not api_key:
    raise ValueError("Pinecone API key is not set. Please ensure the PINECONE_API_KEY environment variable is correct.")

# Pinecone 클라이언트 초기화
pc = Pinecone(api_key=api_key)

# 클라우드 및 리전 설정
cloud = os.getenv('PINECONE_CLOUD', 'aws')
region = os.getenv('PINECONE_REGION', 'us-east-1')
spec = ServerlessSpec(cloud=cloud, region=region)

# 인덱스 이름
index_name = 'example-index'

# 인덱스 객체 얻기
index = pc.Index(index_name)

# 모델 로드
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# 검색 구현
def retrieve_related_tables(query_text, top_k=5):
    query_vector = model.encode(query_text).tolist()
    
    results = index.query(top_k=top_k, vector=query_vector, include_metadata=True)
    
    related_tables = {}
    if results['matches']:
        for match in results['matches']:
            score = match['score']
            metadata = match.get('metadata', {})
            
            sheet_name = metadata.get('sheet')
            table_name = metadata.get('table')
            
            if sheet_name and table_name:
                with open(f'output_tables/{sheet_name}.json', 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    table_data = data.get(table_name, [])

                table_key = f"{sheet_name}_{table_name}"
                related_tables[table_key] = {
                    "sheet": sheet_name,
                    "table": table_name,
                    "data": table_data,
                    "score": score
                }
    
    return related_tables

# JSON을 XML로 변환
def convert_json_to_xml(json_data):
    xml_data = dicttoxml.dicttoxml(json_data, custom_root='root', attr_type=False)
    xml_str = parseString(xml_data).toprettyxml()
    return xml_str

# 예시 사용
query = "일반휴업의 종류를 알려 줘"
related_tables = retrieve_related_tables(query)
if related_tables:
    # LLM에 입력할 데이터
    xml_output = convert_json_to_xml(related_tables)
    print("LLM에 입력할 XML 데이터:")
    print(xml_output)


d:\miniconda3\envs\dev\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
d:\miniconda3\envs\dev\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


LLM에 입력할 XML 데이터:
<?xml version="1.0" ?>
<root>
	<key name="(1) 업무기준_SOP참고자료">
		<sheet>(1) 업무기준</sheet>
		<table>SOP참고자료</table>
		<data>
			<item>
				<metadata>
					<구분>기준관련</구분>
					<열1>-</열1>
					<게시물번호>지식관리&gt; 업무매뉴얼&gt; NEW업무매뉴얼&gt; SC&gt; 주요업무&gt; '휴업'</게시물번호>
				</metadata>
				<text>기준관련 - 지식관리&gt; 업무매뉴얼&gt; NEW업무매뉴얼&gt; SC&gt; 주요업무&gt; '휴업'</text>
			</item>
			<item>
				<metadata>
					<구분/>
					<열1>414653</열1>
					<게시물번호>게시판&gt; 업무공유&gt; SC광장&gt; SC업무공유&gt; '[운영지원팀] '23년 설(구정) 휴업 프로세스 진행안내의 건'</게시물번호>
				</metadata>
				<text>414653 게시판&gt; 업무공유&gt; SC광장&gt; SC업무공유&gt; '[운영지원팀] '23년 설(구정) 휴업 프로세스 진행안내의 건'</text>
			</item>
			<item>
				<metadata>
					<구분/>
					<열1>411702</열1>
					<게시물번호>게시판&gt; 업무공유&gt; SC광장&gt; SC업무공유&gt; '[운영지원팀] '휴업BPM 일부 변경 공유의 건'</게시물번호>
				</metadata>
				<text>411702 게시판&gt; 업무공유&gt; SC광장&gt; SC업무공유&gt; '[운영지원팀] '휴업BPM 일부 변경 공유의 건'</text>
			</item>
		</data>
		<score>6.94720745</score>
	</key>
	<key name="(1) 업무기준_휴업의종류">
		<sheet>(1) 